In [27]:
import pandas as pd

# Load the dataset
df = pd.read_csv('manufacturing_jobs.csv')

In [28]:
# Parse datetime columns
datetime_cols = ['Scheduled_Start', 'Scheduled_End', 'Actual_Start', 'Actual_End']
for col in datetime_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [29]:
# Calculate durations (in hours)
df['Calculated_Scheduled_Duration'] = (df['Scheduled_End'] - df['Scheduled_Start']).dt.total_seconds() / 3600
df['Calculated_Actual_Duration'] = (df['Actual_End'] - df['Actual_Start']).dt.total_seconds() / 3600

In [30]:
# Display basic insights
print("✅ Data loaded and preprocessed successfully.")
print(f"🔍 Total jobs: {len(df)}")
print(f"🛠️ Unique machines: {df['Machine_ID'].nunique()}")
print("\n🧾 Sample preview:")
print(df.head(5))

✅ Data loaded and preprocessed successfully.
🔍 Total jobs: 1000
🛠️ Unique machines: 5

🧾 Sample preview:
  Job_ID Machine_ID Operation_Type  Material_Used  Processing_Time  \
0   J001        M01       Grinding           3.17               76   
1   J002        M01       Grinding           3.35               79   
2   J003        M04       Additive           2.29               56   
3   J004        M04       Grinding           1.76              106   
4   J005        M01          Lathe           1.90               46   

   Energy_Consumption  Machine_Availability     Scheduled_Start  \
0               11.42                    96 2023-03-18 08:00:00   
1                6.61                    84 2023-03-18 08:10:00   
2               11.11                    92 2023-03-18 08:20:00   
3               12.50                    95 2023-03-18 08:30:00   
4                8.13                    88 2023-03-18 08:40:00   

        Scheduled_End        Actual_Start          Actual_End Job_Statu

In [31]:
# Save preprocessed data for further modules
df.to_csv('preprocessed_job_data.csv', index=False)

In [32]:
# Load preprocessed data
df = pd.read_csv('preprocessed_job_data.csv')

In [33]:
# Helper function to find a spare machine
def find_spare_machine(operation_type, excluded_machine):
    candidates = df[
        (df['Operation_Type'] == operation_type) &
        (df['Machine_Availability'] > 90) &
        (df['Machine_ID'] != excluded_machine)
    ]
    if not candidates.empty:
        # Prefer lowest energy consumption
        return candidates.sort_values('Energy_Consumption').iloc[0]['Machine_ID']
    return None

# Initialize results list
scheduled_jobs = []

In [34]:
# Iterate through each job
for index, row in df.iterrows():
    job_id = row['Job_ID']
    original_machine = row['Machine_ID']
    operation = row['Operation_Type']
    duration = row['Processing_Time']
    availability = row['Machine_Availability']
    
    # Schedule Start fallback if missing
    start_time = pd.to_datetime(row['Scheduled_Start']) if pd.notnull(row['Scheduled_Start']) else pd.Timestamp.now()

    if availability > 90:
        status = f"Assigned to {original_machine}"
        end_time = start_time + pd.to_timedelta(duration, unit='h')
        machine_used = original_machine
    else:
        spare_machine = find_spare_machine(operation, original_machine)
        if spare_machine:
            status = f"🔁 Reassigned to spare machine {spare_machine}"
            end_time = start_time + pd.to_timedelta(duration, unit='h')
            machine_used = spare_machine
        else:
            status = f" No available machine for {operation}"
            end_time = None
            machine_used = None

    scheduled_jobs.append({
        'Job_ID': job_id,
        'Assigned_Machine': machine_used,
        'Scheduled_Start': start_time,
        'Estimated_End': end_time,
        'Status': status
    })

In [35]:
# Create a new DataFrame
schedule_df = pd.DataFrame(scheduled_jobs)

# Show results
print("🗓️ Job Scheduling Summary:")
print(schedule_df[['Job_ID', 'Assigned_Machine', 'Scheduled_Start', 'Estimated_End', 'Status']].head(10))

🗓️ Job Scheduling Summary:
  Job_ID Assigned_Machine     Scheduled_Start       Estimated_End  \
0   J001              M01 2023-03-18 08:00:00 2023-03-21 12:00:00   
1   J002              M04 2023-03-18 08:10:00 2023-03-21 15:10:00   
2   J003              M04 2023-03-18 08:20:00 2023-03-20 16:20:00   
3   J004              M04 2023-03-18 08:30:00 2023-03-22 18:30:00   
4   J005              M04 2023-03-18 08:40:00 2023-03-20 06:40:00   
5   J006              M04 2023-03-18 08:50:00 2023-03-22 12:50:00   
6   J007              M03 2023-03-18 09:00:00 2023-03-19 07:00:00   
7   J008              M05 2023-03-18 09:10:00 2023-03-21 16:10:00   
8   J009              M03 2023-03-18 09:20:00 2023-03-20 03:20:00   
9   J010              M01 2023-03-18 09:30:00 2023-03-19 12:30:00   

                              Status  
0                    Assigned to M01  
1  🔁 Reassigned to spare machine M04  
2                    Assigned to M04  
3                    Assigned to M04  
4  🔁 Reassigned to

In [36]:
# Save results for visualization
schedule_df.to_csv('job_schedule_output.csv', index=False)

In [41]:
from collections import defaultdict

# Initialize machine failure history tracker
machine_failure_history = defaultdict(list)
breakdown_alerts = []

# Define threshold values
FAILURE_THRESHOLD = 2  # e.g., 2 consecutive failures
FAILURE_WINDOW = 5
FAILURE_RATIO = 0.6    # e.g., 60% failure rate in recent 5 jobs

# Sort by Scheduled_Start to track in chronological order
df = df.sort_values(by='Scheduled_Start')

# Track failures
for idx, row in df.iterrows():
    machine = row['Machine_ID']
    status = row['Job_Status']
    job_id = row['Job_ID']
    start_time = row['Scheduled_Start']

    # Record status
    machine_failure_history[machine].append(status)

    # Get recent statuses (last 5 jobs max)
    recent_statuses = machine_failure_history[machine][-FAILURE_WINDOW:]

    # Check for consecutive failures
    consecutive_failures = all(s == 'Failed' for s in recent_statuses[-FAILURE_THRESHOLD:])

    # Check failure ratio
    failure_ratio = recent_statuses.count('Failed') / len(recent_statuses)

    # Raise alert if either threshold is met
    if consecutive_failures or failure_ratio >= FAILURE_RATIO:
        alert_msg = f"⚠️ ALERT: {machine} is unreliable! ({failure_ratio*100:.0f}% failures in recent jobs)"
        alert_detail = {
            "Machine_ID": machine,
            "Affected_Job": job_id,
            "Alert_Time": start_time,
            "Alert_Message": alert_msg
        }
        breakdown_alerts.append(alert_detail)

In [42]:
# Convert to DataFrame for display
breakdown_alerts_df = pd.DataFrame(breakdown_alerts)
print("🚨 Breakdown Alerts 🚨")
print(breakdown_alerts_df if not breakdown_alerts_df.empty else "No critical machine issues detected.")

🚨 Breakdown Alerts 🚨
   Machine_ID Affected_Job           Alert_Time  \
0         M04         J003  2023-03-18 08:20:00   
1         M04         J175  2023-03-19 13:00:00   
2         M04         J205  2023-03-19 18:00:00   
3         M03         J222  2023-03-19 20:50:00   
4         M03         J239  2023-03-19 23:40:00   
5         M04         J342  2023-03-20 16:50:00   
6         M01         J514  2023-03-21 21:30:00   
7         M01         J516  2023-03-21 21:50:00   
8         M01         J525  2023-03-21 23:20:00   
9         M01         J527  2023-03-21 23:40:00   
10        M03         J573  2023-03-22 07:20:00   
11        M05         J585  2023-03-22 09:20:00   
12        M05         J591  2023-03-22 10:20:00   
13        M05         J594  2023-03-22 10:50:00   
14        M03         J608  2023-03-22 13:10:00   
15        M03         J609  2023-03-22 13:20:00   
16        M01         J611  2023-03-22 13:40:00   
17        M03         J612  2023-03-22 13:50:00   
18        

In [43]:
!pip install plyer

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
from plyer import notification

# Send alert for each breakdown
for alert in breakdown_alerts:
    notification.notify(
        title="⚠️ Machine Breakdown Alert",
        message=f"{alert['Alert_Message']} | Job: {alert['Affected_Job']} | Time: {alert['Alert_Time']}",
        timeout=10  # seconds
    )


In [52]:
def recommend_replacement(machine_id, job_row, df):
    same_type_machines = df[
        (df['Operation_Type'] == job_row['Operation_Type']) &
        (df['Machine_ID'] != machine_id) &
        (df['Machine_Availability'] > 80)  # arbitrary availability threshold
    ].sort_values(by=['Machine_Availability', 'Energy_Consumption'], ascending=[False, True])

    if not same_type_machines.empty:
        return same_type_machines.iloc[0]['Machine_ID']
    else:
        return "No optimal replacement found"

# Add replacement suggestion to alert
for alert in breakdown_alerts:
    job_row = df[df['Job_ID'] == alert['Affected_Job']].iloc[0]
    replacement = recommend_replacement(alert['Machine_ID'], job_row, df)
    alert['Suggested_Replacement'] = replacement


In [1]:
import pickle
import pandas as pd
from collections import defaultdict
from plyer import notification


class JobSchedulingModel:
    def __init__(self, df):
        self.df = df
        self.machine_failure_history = defaultdict(list)
        self.breakdown_alerts = []

    def send_desktop_alert(self, message):
        notification.notify(
            title="Machine Breakdown Alert",
            message=message,
            timeout=10
        )

    def recommend_replacement(self, machine_id, job_row):
        same_type_machines = self.df[
            (self.df['Operation_Type'] == job_row['Operation_Type']) &
            (self.df['Machine_ID'] != machine_id) &
            (self.df['Machine_Availability'] > 80)  # arbitrary availability threshold
        ].sort_values(by=['Machine_Availability', 'Energy_Consumption'], ascending=[False, True])

        if not same_type_machines.empty:
            return same_type_machines.iloc[0]['Machine_ID']
        else:
            return "No optimal replacement found"

    def track_machine_failures_and_recommendation(self):
        for idx, row in self.df.iterrows():
            machine = row['Machine_ID']
            status = row['Job_Status']
            job_id = row['Job_ID']
            start_time = row['Scheduled_Start']

            # Track failures for the machine
            self.machine_failure_history[machine].append(status)

            recent_statuses = self.machine_failure_history[machine][-5:]
            consecutive_failures = all(s == 'Failed' for s in recent_statuses[-2:])
            failure_ratio = recent_statuses.count('Failed') / len(recent_statuses)

            if consecutive_failures or failure_ratio >= 0.6:
                alert_msg = f"⚠️ ALERT: {machine} is unreliable! ({failure_ratio*100:.0f}% failures)"
                alert_detail = {
                    "Machine_ID": machine,
                    "Affected_Job": job_id,
                    "Alert_Time": start_time,
                    "Alert_Message": alert_msg
                }
                self.breakdown_alerts.append(alert_detail)

                self.send_desktop_alert(alert_msg)

                # Recommend replacement
                job_row = self.df[self.df['Job_ID'] == job_id].iloc[0]
                replacement_machine = self.recommend_replacement(machine, job_row)
                print(f"Recommended replacement for {machine}: {replacement_machine}")

    def save_model(self, filename='job_scheduling_model.pkl'):
        with open(filename, 'wb') as file:
            pickle.dump(self, file)
        print("Model saved successfully!")


# Example usage:

# Assuming df is your preprocessed DataFrame
df = pd.read_csv("preprocessed_job_data.csv")

# Initialize the model
model = JobSchedulingModel(df)

# Track failures and get alerts
model.track_machine_failures_and_recommendation()

# Save the model
model.save_model('job_scheduling_model.pkl')


Recommended replacement for M04: M01
Recommended replacement for M04: M02
Recommended replacement for M04: M03
Recommended replacement for M03: M02
Recommended replacement for M03: M02
Recommended replacement for M04: M01
Recommended replacement for M01: M05
Recommended replacement for M01: M04
Recommended replacement for M01: M03
Recommended replacement for M01: M03
Recommended replacement for M03: M02
Recommended replacement for M05: M02
Recommended replacement for M05: M01
Recommended replacement for M05: M04
Recommended replacement for M03: M02
Recommended replacement for M03: M02
Recommended replacement for M01: M02
Recommended replacement for M03: M02
Recommended replacement for M03: M01
Recommended replacement for M01: M04
Recommended replacement for M01: M04
Recommended replacement for M03: M01
Recommended replacement for M04: M01
Recommended replacement for M02: M01
Recommended replacement for M03: M02
Recommended replacement for M02: M04
Recommended replacement for M02: M03
R

In [2]:
# Save the trained model and its state
model.save_model('job_scheduling_model.pkl')

# Load the saved model
with open('job_scheduling_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model
loaded_model.track_machine_failures_and_recommendation()


Model saved successfully!
Recommended replacement for M04: M02
Recommended replacement for M04: M03
Recommended replacement for M03: M02
Recommended replacement for M03: M02
Recommended replacement for M04: M01
Recommended replacement for M01: M05
Recommended replacement for M01: M04
Recommended replacement for M01: M03
Recommended replacement for M01: M03
Recommended replacement for M03: M02
Recommended replacement for M05: M02
Recommended replacement for M05: M01
Recommended replacement for M05: M04
Recommended replacement for M03: M02
Recommended replacement for M03: M02
Recommended replacement for M01: M02
Recommended replacement for M03: M02
Recommended replacement for M03: M01
Recommended replacement for M01: M04
Recommended replacement for M01: M04
Recommended replacement for M03: M01
Recommended replacement for M04: M01
Recommended replacement for M02: M01
Recommended replacement for M03: M02
Recommended replacement for M02: M04
Recommended replacement for M02: M03
Recommended 

In [1]:
import gradio as gr
import pandas as pd
import plotly.express as px
from datetime import datetime

# Core function
def process_jobs(csv_file):
    df = pd.read_csv(csv_file)

    # Convert date columns
    for col in ['Scheduled_Start', 'Scheduled_End', 'Actual_Start', 'Actual_End']:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Add Calculated Durations if not present
    df['Calculated_Scheduled_Duration'] = (df['Scheduled_End'] - df['Scheduled_Start']).dt.total_seconds() / 3600
    df['Calculated_Actual_Duration'] = (df['Actual_End'] - df['Actual_Start']).dt.total_seconds() / 3600

    # Identify breakdowns: jobs with 'Failed' or NaT in 'Actual_End'
    breakdowns = df[df['Job_Status'].str.contains('Failed|Delayed', na=False)].copy()

    breakdowns['Alert_Message'] = breakdowns.apply(
        lambda row: f"Machine {row['Machine_ID']} failed during Job {row['Job_ID']}", axis=1
    )

    # Recommend replacements
    def recommend_replacement(row):
        same_ops = df[
            (df['Operation_Type'] == row['Operation_Type']) &
            (df['Machine_ID'] != row['Machine_ID']) &
            (df['Machine_Availability'] > 85)
        ].sort_values(by=['Machine_Availability', 'Energy_Consumption'], ascending=[False, True])
        if not same_ops.empty:
            return same_ops.iloc[0]['Machine_ID']
        return "⚠️ No Replacement Found"

    if not breakdowns.empty:
        breakdowns['Suggested_Replacement'] = breakdowns.apply(recommend_replacement, axis=1)

    # Gantt chart: visualize all jobs
    chart_df = df.dropna(subset=['Scheduled_Start', 'Scheduled_End']).copy()
    chart_df['Duration_Hours'] = chart_df['Calculated_Scheduled_Duration']
    chart = px.timeline(chart_df,
                        x_start="Scheduled_Start", x_end="Scheduled_End",
                        y="Machine_ID", color="Job_Status",
                        hover_name="Job_ID", title="📅 Machine Job Schedule Timeline")
    chart.update_yaxes(autorange="reversed")

    # Return table view and chart
    return df, breakdowns[['Job_ID', 'Machine_ID', 'Alert_Message', 'Suggested_Replacement']], chart

# Gradio Interface
app = gr.Interface(
    fn=process_jobs,
    inputs=gr.File(label="Upload Job Schedule CSV"),
    outputs=[
        gr.Dataframe(label="📋 Full Job Schedule"),
        gr.Dataframe(label="🚨 Breakdown Alerts & Replacements"),
        gr.Plot(label="📈 Job Timeline Visualization")
    ],
    title="🔧 Smart Job Scheduler & Breakdown Monitor",
    description="Upload your manufacturing job schedule CSV. Get visual insights, breakdown alerts, and machine recommendations!"
)

app.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
